## What does this script do:

#### Section 1) Converts Suite2p extracted ROI traces and neuropil data output into a numpy or csv file ready for downstream event-related analysis

#### Section 2) Create a pickle file containing a dictionary where each key represents a condition/event-type in the session and the corresponding values are lists containing time stamps of each event in samples

## Section 1

In [ ]:
import numpy as np
import pickle as pkl
import os
import pandas as pd

In [ ]:
"""

USER DEFINE VARIABLES

fdir: string
Directory where the root raw data resides in (not the suite2p output folder)

fname: string
root name of the raw data (without file extensions). Will ultimately dictate the base name of the saved files

"""

fdir = r'D:\olympus_data\Kali\2P Trials for Charles\2022_02_04\fish2_HeatLaser_newHb_30sBaseline_5sLaser_ConcatenatedNoBaseline'
fname = 'fish2_HeatLaser_newHb_30sBaseline_5sLaser_ConcatenatedNoBaseline'

In [ ]:
# define paths for loading s2p data
s2p_dir = os.path.join(fdir, 'suite2p', 'plane0')
s2p_F_path = os.path.join(s2p_dir, 'F.npy')
s2p_Fneu_path = os.path.join(s2p_dir, 'Fneu.npy')
s2p_iscell_path = os.path.join(s2p_dir, 'iscell.npy')
s2p_ops_path = os.path.join(s2p_dir, 'ops.npy')

# define savepaths for converted output data
csv_savepath = os.path.join(fdir, 'suite2p', 'plane0', "{}_s2p_data.csv".format(fname))
npy_savepath = os.path.join(s2p_dir, fname+'_neuropil_corrected_signals.npy')

In [ ]:
# load s2p data
F_data = np.load(s2p_F_path, allow_pickle=True)
Fneu_data = np.load(s2p_Fneu_path, allow_pickle=True)
iscell_data = np.load(s2p_iscell_path, allow_pickle=True)
ops_data = np.load(s2p_ops_path, allow_pickle=True).item()

In [ ]:
npil_corr_signals = F_data - ops_data['neucoeff'] * Fneu_data

iscell_npil_corr_data = npil_corr_signals[iscell_data[:,0].astype('bool'),:]

In [ ]:
# save cell activity data as a csv with ROIs on y axis and samples on x axis
np.save(npy_savepath, iscell_npil_corr_data) # this saves the user-curated neuropil corrected signals as an npy file
pd.DataFrame(data=iscell_npil_corr_data).to_csv(csv_savepath, index=False, header=False) # this saves the same data as a csv file

## Section 2: Make behavioral event timing dictionary

Directions:

1) Adjust the sampling rate (fs) according to your recording parameters

2) Duplicate the line in the 2nd code block below (event_frames line) such that you have the same number of duplicates as you have trial conditions in your session

3) Rename the keys of the dictionary (ie. the string in the first square brackets) to your desired condition names; For example if you have a sucrose condition, replace the 'poke' string with 'sucrose'

4) Update the list on the right side of the equation to reflect the time stamps (in seconds) for the events that occur for that specific condition.

For example if your sucrose deliveries occurred at 10, 15, and 20 seconds after the onset of the recording, and a set of quinine deliveries at 13, 17, 25 you would have a couple of lines looking like this:

`event_frames['sucrose'] = np.rint(np.array([10, 15, 20])*fs)
event_frames['quinine'] = np.rint(np.array([13, 17, 25])*fs)
`

In [ ]:
event_frames = {}
fs = 0.919 # sampling rate

In [ ]:
event_frames['poke'] = np.rint(np.array([30, 60, 90])*fs)
event_frames['drug'] = np.rint(np.array([30, 60, 90])*fs)

In [ ]:
# save dictionary into pickle
with open(os.path.join(s2p_dir, 'framenumberforevents_'+fname+'.pkl'), 'wb') as handle:
    pkl.dump(event_frames, handle, protocol=4) #pkl.HIGHEST_PROTOCOL